# Batch inference with Gemma/PaliGemma with HF + GCP

Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models, developed by Google DeepMind and other teams across Google. Text Generation Inference (TGI) is a toolkit developed by Hugging Face for deploying and serving LLMs, with high performance text generation. And, Google Vertex AI is a Machine Learning (ML) platform that lets you train and deploy ML models and AI applications, and customize large language models (LLMs) for use in your AI-powered applications. This example showcases how to deploy any supported text-generation model, in this case [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it), from the Hugging Face Hub on Vertex AI using the TGI DLC available in Google Cloud Platform (GCP).

![`google/gemma-7b-it` in the Hugging Face Hub](./assets/model-in-hf-hub.png)

## Setup / Configuration

First, you need to install `gcloud` in your local machine, which is the command-line tool for Google Cloud, following the instructions at [Cloud SDK Documentation - Install the gcloud CLI](https://cloud.google.com/sdk/docs/install).

Then, you also need to install the `google-cloud-aiplatform` Python SDK, required to programmatically create the Vertex AI model, register it, acreate the endpoint, and deploy it on Vertex AI.

In [60]:
!pip install --upgrade google-cloud-aiplatform


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


Optionally, to ease the usage of the commands within this tutorial, you need to set the following environment variables for GCP:

In [61]:
%env PROJECT_ID=multimodal-representations
%env LOCATION=us-central1
%env CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310:latest

env: PROJECT_ID=multimodal-representations
env: LOCATION=us-central1
env: CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310:latest


Then you need to login into your GCP account and set the project ID to the one you want to use to register and deploy the models on Vertex AI.

In [62]:
!gcloud auth login
!gcloud config set project $PROJECT_ID

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=SVZIOf0TwZijRq9vSPjOY4pYUZeVWy&access_type=offline&code_challenge=oOEdflrwYAWtdPpMJ7vZZWXkt6Ro1wXJJisvyihidJc&code_challenge_method=S256


You are now logged in as [daliumuwork@gmail.com].
Your current project is [multimodal-representations].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


Once you are logged in, you need to enable the necessary service APIs in GCP, such as the Vertex AI API, the Compute Engine API, and Google Container Registry related APIs.

**Warning:** Make sure, manually, that these are disabled after running exps (even though we will explicitly write code to disable them)

In [63]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable container.googleapis.com
!gcloud services enable containerregistry.googleapis.com
!gcloud services enable containerfilesystem.googleapis.com

Operation "operations/acf.p2-841337720906-482ed210-3ed3-4346-af2c-66d0628a0a49" finished successfully.
Operation "operations/acat.p2-841337720906-20969f9c-87d2-478f-ab22-7a2ff3ed73f3" finished successfully.


## Register model on Vertex AI

Once everything is set up, you can already initialize the Vertex AI session via the `google-cloud-aiplatform` Python SDK as follows:

In [64]:
import os
from google.cloud import aiplatform

aiplatform.init(
    project=os.getenv("PROJECT_ID"),
    location=os.getenv("LOCATION"),
)

Since Gemma models are gated, you need to login into your Hugging Face Hub account with a read-access token either fine-grained with access to the gated model, or just overall read-access to your account. More information on how to generate a read-only access token for the Hugging Face Hub in the instructions at <https://huggingface.co/docs/hub/en/security-tokens>.

In [65]:
!pip install --upgrade --quiet huggingface_hub


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [66]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to /home/dali/.cache/huggingface/token
Login successful


Then you can already "upload" the model i.e. register the model on Vertex AI. It is not an upload per se, since the model will be automatically downloaded from the Hugging Face Hub in the Hugging Face DLC for TGI on startup via the `MODEL_ID` environment variable, so what is uploaded is only the configuration, not the model weights.

Before going into the code, let's quickly review the arguments provided to the `upload` method:

* **`display_name`** is the name that will be shown in the Vertex AI Model Registry.

* **`serving_container_image_uri`** is the location of the Hugging Face DLC for TGI that will be used for serving the model.

* **`serving_container_environment_variables`** are the environment variables that will be used during the container runtime, so these are aligned with the environment variables defined by `text-generation-inference`, which are analog to the [`text-generation-launcher` arguments](https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/launcher). Additionally, the Hugging Face DLCs for TGI also capture the `AIP_` environment variables from Vertex AI as in [Vertex AI Documentation - Custom container requirements for prediction](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements).

    * `MODEL_ID` is the identifier of the model in the Hugging Face Hub. To explore all the supported models you can check <https://huggingface.co/models?sort=trending&other=text-generation-inference>.
    * `NUM_SHARD` is the number of shards to use if you don't want to use all GPUs on a given machine e.g. if you have two GPUs but you just want to use one for TGI then `NUM_SHARD=1`, otherwise it matches the `CUDA_VISIBLE_DEVICES`.
    * `MAX_INPUT_TOKENS` is the maximum allowed input length (expressed in number of tokens), the larger it is, the larger the prompt can be, but also more memory will be consumed.
    * `MAX_TOTAL_TOKENS` is the most important value to set as it defines the "memory budget" of running clients requests, the larger this value, the larger amount each request will be in your RAM and the less effective batching can be.
    * `MAX_BATCH_PREFILL_TOKENS` limits the number of tokens for the prefill operation, as it takes the most memory and is compute bound, it is interesting to limit the number of requests that can be sent.
    * `HUGGING_FACE_HUB_TOKEN` is the Hugging Face Hub token, required as [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) is a gated model.

* (optional) **`serving_container_ports`** is the port where the Vertex AI endpoint will be exposed, by default 8080.

For more information on the supported `aiplatform.Model.upload` arguments, check its Python reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_upload.

In [67]:
print(os.getenv("CONTAINER_URI"))
#huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310

us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310:latest


In [68]:
!gcloud auth configure-docker us-docker.pkg.dev


{
  "credHelpers": {
    "us-docker.pkg.dev": "gcloud",
    "gcr.io": "gcloud"
  }
}
Adding credentials for: us-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [69]:
from huggingface_hub import get_token

HF_MODEL = "google/paligemma-3b-pt-224" # change this
DISPLAY_NAME = "paligemma-3b-pt-224"

model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    serving_container_image_uri=os.getenv("CONTAINER_URI"),
    serving_container_environment_variables={
        "MODEL_ID": HF_MODEL,
        "NUM_SHARD": "1",
        "MAX_INPUT_TOKENS": "512", # I am sure these can be further optimized
        "MAX_TOTAL_TOKENS": "1024",
        "MAX_BATCH_PREFILL_TOKENS": "1512",
        "HUGGING_FACE_HUB_TOKEN": get_token(),
    },
    serving_container_ports=[8080],
)
model.wait()

Creating Model
Create Model backing LRO: projects/841337720906/locations/us-central1/models/1775620019393134592/operations/1035206312268398592
Model created. Resource name: projects/841337720906/locations/us-central1/models/1775620019393134592@1
To use this Model in another session:
model = aiplatform.Model('projects/841337720906/locations/us-central1/models/1775620019393134592@1')


![Model on Vertex AI Model Registry](./assets/vertex-ai-model.png)

## Deploy model on Vertex AI

After the model is registered on Vertex AI, you need to define the endpoint that you want to deploy the model to, and then link the model deployment to that endpoint resource.

To do so, you need to call the method `aiplatform.Endpoint.create` to create a new Vertex AI endpoint resource (which is not linked to a model or anything usable yet).

In [70]:
endpoint = aiplatform.Endpoint.create(display_name=f"{DISPLAY_NAME}-endpoint")

Creating Endpoint
Create Endpoint backing LRO: projects/841337720906/locations/us-central1/endpoints/7248140079485419520/operations/6034764848603070464
Endpoint created. Resource name: projects/841337720906/locations/us-central1/endpoints/7248140079485419520
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/841337720906/locations/us-central1/endpoints/7248140079485419520')


![Vertex AI Endpoint created](./assets/vertex-ai-endpoint.png)

Now you can deploy the registered model in an endpoint on Vertex AI.

The `deploy` method will link the previously created endpoint resource with the model that contains the configuration of the serving container, and then, it will deploy the model on Vertex AI in the specified instance.

Before going into the code, let's quickly review the arguments provided to the `deploy` method:

- **`endpoint`** is the endpoint to deploy the model to, which is optional, and by default will be set to the model display name with the `_endpoint` suffix.
- **`machine_type`**, **`accelerator_type`** and **`accelerator_count`** are arguments that define which instance to use, and additionally, the accelerator to use and the number of accelerators, respectively. The `machine_type` and the `accelerator_type` are tied together, so you will need to select an instance that supports the accelerator that you are using and vice-versa. More information about the different instances at [Compute Engine Documentation - GPU machine types](https://cloud.google.com/compute/docs/gpus), and about the `accelerator_type` naming at [Vertex AI Documentation - MachineSpec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec).

For more information on the supported `aiplatform.Model.deploy` arguments, you can check its Python reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_deploy.

In [149]:
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

Deploying model to Endpoint : projects/841337720906/locations/us-central1/endpoints/7248140079485419520


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/841337720906/locations/us-central1/endpoints/7248140079485419520


Deploy Endpoint model backing LRO: projects/841337720906/locations/us-central1/endpoints/7248140079485419520/operations/7278286111338659840


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/841337720906/locations/us-central1/endpoints/7248140079485419520/operations/7278286111338659840


Endpoint model deployed. Resource name: projects/841337720906/locations/us-central1/endpoints/7248140079485419520


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/841337720906/locations/us-central1/endpoints/7248140079485419520


**WARNING**: _The Vertex AI endpoint deployment via the `deploy` method may take from 15 to 25 minutes._

## Online predictions on Vertex AI

Finally, you can run the online predictions on Vertex AI using the `predict` method, which will send the requests to the running endpoint in the `/predict` route specified within the container following Vertex AI I/O payload formatting.

As you are serving a `text-generation` model, you will need to make sure that the chat template, if any, is applied correctly to the input conversation; meaning that `transformers` need to be installed so as to instantiate the `tokenizer` for [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) and run the `apply_chat_template` method over the input conversation before sending the input within the payload to the Vertex AI endpoint.

**Note:** The chat template might not be needed for Gemma-2b/the model PaliGemma is based on.

In [72]:
!pip install --upgrade --quiet transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


After the installation is complete, the following snippet will apply the chat template to the conversation:

>**This isn't needed for PaliGemma but might be needed for Gemma-2b/the LM it is based on.**


### Via Python

#### Within the same session

If you are willing to run the online prediction within the current session, you can send requests programmatically via the `aiplatform.Endpoint` (returned by the `aiplatform.Model.deploy` method) as in the following snippet:


Here, we have listed code to supply images to the model via two different ways: (1) via urls, and (2) from local files. KM has extrapolated this syntax from the TGI docs on huggingface: https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/visual_language_models

In [73]:
import base64
import requests
import io

**URL Based**

In [150]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/rabbit.png"

output = deployed_model.predict(
    instances=[
        {
            "inputs":"![]{url}What is the animal wearing?",
            "parameters":{"max_new_tokens": 100, "do_sample": False}
        }
    ]
)
# > helmet

In [151]:
output

Prediction(predictions=['clothes'], deployed_model_id='9198981570415820800', metadata=None, model_version_id='1', model_resource_name='projects/841337720906/locations/us-central1/models/1775620019393134592', explanations=None)

**Locally**

In [142]:
!pwd

/home/dali/work/postdoc/multimodal-semantic-signals/multimodal-representations/notebooks


In [155]:
image_path = "fgqa_hs/images/2333016.png"

with open(image_path, "rb") as f:
    image = base64.b64encode(f.read()).decode("utf-8")

image = f"data:image/png;base64,{image}"

output = deployed_model.predict(
    instances=[
        {
            "inputs":f"![]({image})What is the animal wearing?",
            "parameters":{"max_new_tokens": 100, "do_sample": False}
        }
    ]
)
#> space suit

In [76]:
# is different when image is passed from a local file vs. from an url
# even when we use greedy decoding (I think?), strange!
output

Prediction(predictions=['suit'], deployed_model_id='9198981570415820800', metadata=None, model_version_id='1', model_resource_name='projects/841337720906/locations/us-central1/models/1775620019393134592', explanations=None)

Producing the following `output`:

```
Prediction(predictions=['space suit'], deployed_model_id='6484700777808920576', metadata=None, model_version_id='1', model_resource_name='projects/20178026/locations/us-central1/models/1030014796319162368', explanations=None)
```

In [133]:
def predict_answers(model, images, questions, gen_config):
    assert len(images) == len(questions)

    instances = []

    #generation_config = {
    #    "max_new_tokens": 256,
    #    "do_sample": True,
    #    "top_p": 0.2,
    #    "temperature": 0.2,
    #}

    
    binarized_images = []
    for i, image in enumerate(images):
        img = f"data:image/png;base64,{image}"
        instance = {"inputs":f"![]({img}){questions[i]}",
                "parameters": gen_config}
        instances.append(instance)
    print(len(instances[0]["inputs"]))
    output = model.predict(
        instances=instances
    )
    return output

In [134]:
image_path = "/home/dali/Downloads/rabbit.png"

generation_config = {"max_new_tokens": 100, "do_sample": False}

with open(image_path, "rb") as f:
    image = base64.b64encode(f.read()).decode("utf-8")
    images = [image, image]
    output = predict_answers(deployed_model, 
                             images, 
                             ["What is the animal wearing?", "Is the animal in the image threatening?"], 
                             generation_config)
for prediction in output.predictions:
    print(prediction)
print(output)
#> space suit

561586
helmet
yes y preds threatens alos garden
Prediction(predictions=['helmet', 'yes y preds threatens alos garden'], deployed_model_id='9198981570415820800', metadata=None, model_version_id='1', model_resource_name='projects/841337720906/locations/us-central1/models/1775620019393134592', explanations=None)


## Evaluation task

Based on [https://huggingface.co/docs/google-cloud/main/examples/vertex-ai-notebooks-evaluate-llms-with-vertex-ai](https://huggingface.co/docs/google-cloud/main/examples/vertex-ai-notebooks-evaluate-llms-with-vertex-ai)

In [222]:
from datasets import load_dataset

dataset = load_dataset("fgqa_hs", split='test')

dataset

Dataset({
    features: ['question', 'answer', 'argument', 'image', 'substitutions'],
    num_rows: 11701
})

In [223]:
dataset[0]

{'question': 'Is the curtain on the right side or on the left of the picture?',
 'answer': 'right',
 'argument': 'curtain',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x334>,
 'substitutions': {'hypernym': ['blind',
   'protective covering',
   'covering',
   'cloth'],
  'question': ['Is the blind on the right side or on the left of the picture?',
   'Is the protective covering on the right side or on the left of the picture?',
   'Is the covering on the right side or on the left of the picture?',
   'Is the cloth on the right side or on the left of the picture?']}}

We must convert to a pandas dataset in order to use the Vertex Evaluation API

In [224]:
df = dataset.to_pandas()

In [231]:
df['image'][0]

{'bytes': None,
 'path': '/home/dali/.cache/huggingface/datasets/downloads/extracted/9d8531eb1664568201094473c8dc315fc8c009a6bb87849aacd669850d60ef87/2333016.jpg'}

In [225]:
def binarize_image_old(image, img_type='jpg'):
    with open(f"{path_prefix}{img_id}.{img_type}", "rb") as f:
        image = base64.b64encode(f.read()).decode("utf-8")
        return f"data:image/{img_type};base64,{image}"

In [234]:

from io import BytesIO
def binarize_image(img_path, img_type='jpg'):
    
    with open(f"{img_path}, "rb") as f:
        image = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/{img_type};base64,{image}"

SyntaxError: unterminated string literal (detected at line 4) (376898519.py, line 4)

In [233]:
binarize_image(df['image'][0]['path'])

AttributeError: 'str' object has no attribute 'save'

Step 0. Convert images to base64 representations

In [194]:
df["img"] = df["imageId"].apply(lambda x: binarize_image(x))

In [195]:
df['prompt'] = df.apply(lambda row: f"![]({row['img']})Answer the following question about the given image. Answer with a single concept: {row['question']}", axis=1)
df['prompt']

0     ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
1     ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
2     ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
3     ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
4     ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
                            ...                        
95    ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
96    ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
97    ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEA...
98    ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEB...
99    ![](data:image/jpg;base64,/9j/4AAQSkZJRgABAQEB...
Name: prompt, Length: 100, dtype: object

In [196]:
df['reference'] = df['answer']

Drop all columns that we do not need for the prediction task

In [215]:
df = df[['prompt','reference']]

In [216]:
from vertexai.evaluation import EvalTask
# 2. create eval task
eval_task = EvalTask(
        dataset=df,
        metrics=["exact_match", "rouge"],
        experiment="multimodal-hypernym-semantics",
)

In [217]:
def generate(prompt, generation_config=generation_config):
    payload = prompt_to_payload(prompt, generation_config)
    output = deployed_model.predict(instances=[payload])
    generated_text = output.predictions[0]
    return generated_text

def prompt_to_payload(prompt, generation_config):
    return {"inputs": prompt, "parameters": generation_config}

In [218]:
generate(df['prompt'][0])

'right'

In [198]:
import uuid
# 3. run eval task
# Note: If the last iteration takes > 1 minute you might need to retry the evaluation
exp_results = eval_task.evaluate(
        model=generate, experiment_run_name=f"test-gqa-{str(uuid.uuid4())[:8]}"
)

Associating projects/841337720906/locations/us-central1/metadataStores/default/contexts/multimodal-hypernym-semantics-test-gqa-ebfeb9dd to Experiment: multimodal-hypernym-semantics


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/841337720906/locations/us-central1/metadataStores/default/contexts/multimodal-hypernym-semantics-test-gqa-ebfeb9dd to Experiment: multimodal-hypernym-semantics


Generating a total of 100 responses from the custom model function.


INFO:vertexai.evaluation._evaluation:Generating a total of 100 responses from the custom model function.
  0%|                                                                                                                                                                                                                                                                                                                                | 0/100 [00:00<?, ?it/s]

132997
198489
172737
234698
127630


...

INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 200 Vertex Gen AI Evaluation Service API requests.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [13:16<00:00,  3.98s/it]

All 200 metric requests are successfully computed.



INFO:vertexai.evaluation._evaluation:All 200 metric requests are successfully computed.


Evaluation Took:796.4613361860393 seconds


INFO:vertexai.evaluation._evaluation:Evaluation Took:796.4613361860393 seconds


In [202]:
print(exp_results.summary_metrics)
print(f"{exp_results.summary_metrics['exact_match/mean']}")
results["test"] = exp_results.summary_metrics["exact_match/mean"]

for prompt_name, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{prompt_name}: {score}")

{'row_count': 100, 'exact_match/mean': np.float64(0.28), 'exact_match/std': np.float64(0.4512608598542129), 'rouge/mean': np.float64(0.290666667), 'rouge/std': np.float64(0.451213105135993)}
0.28
test: 0.28


### Check predictions

In [213]:
exp_results.metrics_table[['question','reference','response']]

,question,reference,response
0,Is the curtain on the right side or on the lef...,right,right
1,What is located on top of the coffee table?,book,goldgold bara tier tier
2,Is the surfer to the left or to the right of t...,right,right orson
3,Which kind of fast food is the bacon on?,pizza,food
4,Is the boat on the right of the picture?,no,no
...,...,...,...
95,Is there any bottle on the table?,yes,no
96,Do you see white breads?,yes,no
97,Which color is the skateboard in the bottom?,brown,blue
98,Is the orange cat sitting on a desk?,no,no


# Batch inference

Below is example code from the GCP documentation found at (https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions)[https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions]

Also check (https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/batch_eval_llm.ipynb)[https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/batch_eval_llm.ipynb]
    

In [ ]:
def create_batch_prediction_job_dedicated_resources_sample(
    model,
    job_display_name: str,
    gcs_source,
    gcs_destination: str,
    machine_type="g2-standard-4", #$0.8129 USD / hour
    accelerator_type="NVIDIA_L4", #$0.644046 USD / hour
    accelerator_count=1,
    instances_format: str = "jsonl",
    starting_replica_count: int = 1,
    max_replica_count: int = 1,
    sync: bool = True,
):


    batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        gcs_source=gcs_source,
        gcs_destination_prefix=gcs_destination,
        instances_format=instances_format,
        starting_replica_count=starting_replica_count,
        max_replica_count=max_replica_count,
        
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        sync=sync,
    )

    batch_prediction_job.wait()

    print(batch_prediction_job.display_name)
    print(batch_prediction_job.resource_name)
    print(batch_prediction_job.state)
    return batch_prediction_job

In [ ]:
batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        gcs_source=gcs_source,
        gcs_destination_prefix=gcs_destination,
        instances_format=instances_format,
        machine_type=machine_type,
        accelerator_count=accelerator_count,
        accelerator_type=accelerator_type,
        starting_replica_count=starting_replica_count,
        max_replica_count=max_replica_count,
        sync=sync,
)


## Resource clean-up (DEFINITELY DO THIS)

Finally, you can already release the resources that you've created as follows, to avoid unnecessary costs:

* `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
* `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully, after the `undeploy_all` method.
* `model.delete` to delete the model from the registry.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

Alternatively, you can also remove those from the Google Cloud Console following the steps:

* Go to Vertex AI in Google Cloud
* Go to Deploy and use -> Online prediction
* Click on the endpoint and then on the deployed model/s to "Undeploy model from endpoint"
* Then go back to the endpoint list and remove the endpoint
* Finally, go to Deploy and use -> Model Registry, and remove the model

In [ ]:
# Disable APIs

!gcloud services disable aiplatform.googleapis.com
!gcloud services disable compute.googleapis.com
!gcloud services disable container.googleapis.com
!gcloud services disable containerregistry.googleapis.com
!gcloud services disable containerfilesystem.googleapis.com

### PLEASE ALSO MANUALLY ENSURE ALL APIS ARE DISABLED ON GCP AFTER THIS IS DONE!
